In [1]:
using Pkg
# use saved environment with compatible dependencies. This notebook does not work with new
# versions of Nemo
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `d:\SeqFISH+Processing_project\UntanglingBarcodes\codebook_generation\get_RS_codebooks\local`
Precompiling packages...
   1302.8 ms  ✓ FLINT_jll
   1007.5 ms  ✓ Antic_jll
   1114.6 ms  ✓ Arb_jll
   2582.5 ms  ✓ NVTX
   2715.8 ms  ✓ GPUArrays
   1162.6 ms  ✓ Calcium_jll
  23165.2 ms  ✓ Nemo
  52204.0 ms  ✓ CUDA
   5608.3 ms  ✓ Atomix → AtomixCUDAExt
  9 dependencies successfully precompiled in 65 seconds. 86 already precompiled.


In [2]:
using Nemo

# Introduction

This notebook shows how to generate codebooks for seqFISH experiments using [Reed-Solomon Codes](https://en.wikipedia.org/wiki/Reed%E2%80%93Solomon_error_correction). To run the google colab version of this notebook, click [here](https://colab.research.google.com/github/CaiGroup/UntanglingBarcodes/blob/master/codebook_generation/get_RS_codebooks/colab/gen_RS_q11k7_code.jl.ipynb). Reed-Solomon are part of a special class of error-correcting codes called [Maximum Distance Separable codes](https://en.wikipedia.org/wiki/Singleton_bound#MDS_codes) (MDS code) which achieve equality in the [Singleton bound](https://en.wikipedia.org/wiki/Singleton_bound). This means that MDS codes acheive the maximum possible extra difference between their codewords from every redundant parity check symbol, and gain the the most possible robustless to error for the increased cost of encoding information with more symbols.

The number of codewords of a given weight in an MDS code weight is given by the the expression 

$(q-1)\binom{w}{n}\sum_{i=0}^{w-d}(-1)^i \binom{i}{w-1}q^{w-d-i}$

(Macwilliams and Sloan)


## Set Reed-Solomon code parameters

In [3]:
q = 11
k = 7

7

In [4]:
function def_RS_code(_q :: Int64, deg :: Int64, nmk :: Int64)
    global q = _q
    global k = (q-1) - nmk
    F, α = FiniteField(q, deg, "α")
    R, x = PolynomialRing(F, "x")
    RR =  ResidueRing(R, x^(q^deg-1)-1)
    gp = 1
    for i = 1:nmk
        gp = gp*RR(x - α^i)
    end
    return RR, R, gp, x, α
end


function cvt_fq_nmod_2_int(x::fq_nmod)
    if iszero(x)
        return 0
    end
    for i = 1:(q-1)
        if iszero(i+x)
            return q-i
        end
    end 
end

function get_cw_array(cw)
    coeffs = Array{Union{Int8, Nothing}}(nothing, q-1)
    for i = 0:(q-2) #9
        coeffs[i+1] = cvt_fq_nmod_2_int(coeff(cw.data,i))
    end
    
    return coeffs
end


RR, R, gp, x, α = def_RS_code(q,1,q-1-k)

(Residue ring of univariate polynomial ring modulo x^10 + 10, Univariate polynomial ring in x over GF(11), x^3 + 8*x^2 + x + 2, x, 2)

We can pring the generating polynomial

In [5]:
gp

x^3 + 8*x^2 + x + 2

In [6]:
# function convert codeword from specialized math objects to an Int Array
get_cw(i) = get_cw_array(gp*R(collect(Tuple(i)))) 

get_cw (generic function with 1 method)

In [7]:
@time q11_k7 = map(get_cw, CartesianIndices(Tuple(fill(q,k))));

1237.163352 seconds (3.94 G allocations: 188.679 GiB, 29.27% gc time, 0.03% compilation time)


In [8]:
length(q11_k7)

19487171

In [9]:
q11_k7 = reshape(q11_k7, length(q11_k7))

19487171-element Vector{Vector{Union{Nothing, Int8}}}:
 [2, 3, 0, 1, 1, 1, 1, 10, 9, 1]
 [4, 4, 8, 2, 1, 1, 1, 10, 9, 1]
 [6, 5, 5, 3, 1, 1, 1, 10, 9, 1]
 [8, 6, 2, 4, 1, 1, 1, 10, 9, 1]
 [10, 7, 10, 5, 1, 1, 1, 10, 9, 1]
 [1, 8, 7, 6, 1, 1, 1, 10, 9, 1]
 [3, 9, 4, 7, 1, 1, 1, 10, 9, 1]
 [5, 10, 1, 8, 1, 1, 1, 10, 9, 1]
 [7, 0, 9, 9, 1, 1, 1, 10, 9, 1]
 [9, 1, 6, 10, 1, 1, 1, 10, 9, 1]
 ⋮
 [6, 3, 2, 3, 0, 0, 0, 0, 0, 0]
 [8, 4, 10, 4, 0, 0, 0, 0, 0, 0]
 [10, 5, 7, 5, 0, 0, 0, 0, 0, 0]
 [1, 6, 4, 6, 0, 0, 0, 0, 0, 0]
 [3, 7, 1, 7, 0, 0, 0, 0, 0, 0]
 [5, 8, 9, 8, 0, 0, 0, 0, 0, 0]
 [7, 9, 6, 9, 0, 0, 0, 0, 0, 0]
 [9, 10, 3, 10, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Let's count how many codewords there are of each weight. Compare to the output of Make_RS_Code_Table_expanded.jl

In [10]:
weights = [sum(map(cw -> sum(.~iszero.(cw)), q11_k7) .== w) for w in 0:10]
weights

11-element Vector{Int64}:
       1
       0
       0
       0
    2100
   17640
  159600
  900000
 3381750
 7512900
 7513180

In [11]:
cb = []
weights_en = [1,0,0,0,0,0,0,0,0,0,0]

w4_cws = []
w5_cws = []
w6_cws = []
niters = 0
li1 = -1
for cw in q11_k7 #CartesianIndices(Tuple(fill(q,k)))
    weight = sum(cw .!= 0)
    if weight == 4
        push!(w4_cws, cw)
    end
    if weight == 5
        push!(w5_cws, cw)
    end
    if weight == 6
        w6_cws = []
    end
    for w = 3:10
        if weight == w
            weights_en[w+1] += 1
        end
    end
end

In [12]:
using DelimitedFiles
open("output/RS_q"*string(q)*"_k"*string(k)*"_w4cb.csv", "w") do io
    writedlm(io, hcat(w4_cws...)',",")
end

In [13]:
using DelimitedFiles
open("output/RS_q"*string(q)*"_k"*string(k)*"_w5cb.csv", "w") do io
    writedlm(io, hcat(w5_cws...)',",")
end

In [14]:
using DelimitedFiles
open("output/RS_q"*string(q)*"_k"*string(k)*"_w6cb.csv", "w") do io
    writedlm(io, hcat(w6_cws...)',",")
end

In [15]:
w4_cws

2100-element Vector{Any}:
 Union{Nothing, Int8}[1, 0, 0, 4, 0, 0, 0, 0, 9, 1]
 Union{Nothing, Int8}[0, 0, 10, 0, 3, 0, 0, 0, 9, 1]
 Union{Nothing, Int8}[0, 4, 0, 0, 0, 0, 0, 2, 9, 1]
 Union{Nothing, Int8}[5, 0, 9, 0, 0, 0, 0, 0, 10, 1]
 Union{Nothing, Int8}[0, 3, 0, 3, 0, 0, 0, 0, 10, 1]
 Union{Nothing, Int8}[0, 0, 0, 0, 10, 9, 0, 0, 10, 1]
 Union{Nothing, Int8}[0, 1, 0, 0, 0, 0, 7, 4, 0, 1]
 Union{Nothing, Int8}[0, 0, 0, 7, 0, 2, 0, 4, 0, 1]
 Union{Nothing, Int8}[0, 9, 0, 0, 8, 0, 0, 5, 0, 1]
 Union{Nothing, Int8}[0, 0, 7, 0, 0, 3, 0, 5, 0, 1]
 ⋮
 Union{Nothing, Int8}[4, 2, 5, 2, 0, 0, 0, 0, 0, 0]
 Union{Nothing, Int8}[6, 3, 2, 3, 0, 0, 0, 0, 0, 0]
 Union{Nothing, Int8}[8, 4, 10, 4, 0, 0, 0, 0, 0, 0]
 Union{Nothing, Int8}[10, 5, 7, 5, 0, 0, 0, 0, 0, 0]
 Union{Nothing, Int8}[1, 6, 4, 6, 0, 0, 0, 0, 0, 0]
 Union{Nothing, Int8}[3, 7, 1, 7, 0, 0, 0, 0, 0, 0]
 Union{Nothing, Int8}[5, 8, 9, 8, 0, 0, 0, 0, 0, 0]
 Union{Nothing, Int8}[7, 9, 6, 9, 0, 0, 0, 0, 0, 0]
 Union{Nothing, Int8}[9, 10,

In [16]:
H = [cvt_fq_nmod_2_int((α^(i[1]))^i[2]) for i in CartesianIndices((q-k-1, q-1))]
H

3×10 Matrix{Int64}:
 2  4  8  5  10  9  7  3  6  1
 4  5  9  3   1  4  5  9  3  1
 8  9  6  4  10  3  2  5  7  1

In [17]:
all(map(cw -> all(iszero.(H*cw .% q)), w4_cws))

true

In [18]:
all(map(cw -> all(iszero.(H*cw .% q)), w5_cws))

true

In [19]:
all(map(cw -> all(iszero.(H*cw .% q)), w6_cws))

true

In [20]:
open("output/RS_q"*string(q)*"_k"*string(k)*"_H.csv", "w") do io
    writedlm(io, H,",")
end